In [4]:
import pandas as pd
import os
from typing import Optional

def fusionner_csv_par_colonnes_robustes(
    fichier_gauche_path: str,
    fichier_droit_path: str,
    colonne_cle: str,
    type_jointure: str = 'inner'
) -> Optional[pd.DataFrame]:
    """
    Fusionne deux fichiers CSV par une colonne clé avec gestion d'erreurs.
    Vérifie la présence des fichiers, l'existence de la clé, et le format de la clé.
    
    Args:
        fichier_gauche_path: Chemin du premier fichier CSV.
        fichier_droit_path: Chemin du second fichier CSV.
        colonne_cle: Nom de la colonne à utiliser pour la jointure (ex: 'time').
        type_jointure: Type de jointure ('inner', 'left', 'right', 'outer').
                       
    Returns:
        Le DataFrame fusionné ou None en cas d'erreur critique.
    """
    
    # Paramètres de lecture basés sur votre format habituel
    params_lecture = {'sep': ';', 'decimal': ',', 'parse_dates': [colonne_cle]}
    
    # ------------------ 1. Gestion des erreurs de lecture/format ------------------
    try:
        if not os.path.exists(fichier_gauche_path) or not os.path.exists(fichier_droit_path):
            print("ERREUR CRITIQUE : Un ou plusieurs chemins de fichiers sont introuvables.")
            return None
            
        df_gauche = pd.read_csv(fichier_gauche_path, **params_lecture)
        df_droit = pd.read_csv(fichier_droit_path, **params_lecture)
        
    except Exception as e:
        print(f"ERREUR CRITIQUE lors de la lecture des fichiers (vérifiez le chemin ou les paramètres sep/decimal) : {e}")
        return None

    # ------------------ 2. Gestion des erreurs de colonne clé ------------------
    if colonne_cle not in df_gauche.columns:
        print(f"ERREUR : La colonne clé '{colonne_cle}' est absente du fichier de gauche.")
        return None
    if colonne_cle not in df_droit.columns:
        print(f"ERREUR : La colonne clé '{colonne_cle}' est absente du fichier de droite.")
        return None
        
    # ------------------ 3. Gestion des erreurs de format de clé (Datetime) ------------------
    # On vérifie que la colonne clé est de type datetime (après le parse_dates)
    if not pd.api.types.is_datetime64_any_dtype(df_gauche[colonne_cle]):
        print(f"ATTENTION : La colonne clé '{colonne_cle}' du fichier de gauche n'est pas au format Datetime.")
        # Tentative de conversion si la lecture a échoué silencieusement
        try:
            df_gauche[colonne_cle] = pd.to_datetime(df_gauche[colonne_cle])
        except:
            print(f"ERREUR : Impossible de convertir la clé '{colonne_cle}' du fichier de gauche en format Date/Heure.")
            return None

    if not pd.api.types.is_datetime64_any_dtype(df_droit[colonne_cle]):
        print(f"ATTENTION : La colonne clé '{colonne_cle}' du fichier de droite n'est pas au format Datetime.")
        try:
            df_droit[colonne_cle] = pd.to_datetime(df_droit[colonne_cle])
        except:
            print(f"ERREUR : Impossible de convertir la clé '{colonne_cle}' du fichier de droite en format Date/Heure.")
            return None

    # ------------------ 4. Fusion des DataFrames ------------------
    try:
        df_fusionne = pd.merge(
            df_gauche,
            df_droit,
            on=colonne_cle,
            how=type_jointure,
            suffixes=('_gauche', '_droit')
        )
        print(f"Fusion réussie avec la jointure '{type_jointure}' sur la clé '{colonne_cle}'.")
        return df_fusionne
        
    except Exception as e:
        print(f"ERREUR : Échec de l'opération pd.merge : {e}")
        return None

In [5]:
fusionner_csv_par_colonnes_robustes(
    "df_162_clean.csv",
    "csv_meteo.csv",
    "datetime",
    'inner'
)

ERREUR CRITIQUE : Un ou plusieurs chemins de fichiers sont introuvables.


In [38]:
def merge_meteo(df_velo, df_meteo):
    df = df_velo.merge(df_meteo, on="datetime", how="outer")
    return df

In [41]:
df_meteo = pd.read_csv('api_meteo.csv')
df_velo = pd.read_csv('api_velo.csv')

# --- Étape 0 : Assurer que les colonnes existent et sont du type 'object' ou 'string' ---
df_meteo['datetime'] = df_meteo['datetime'].astype(str)
df_velo['datetime'] = df_velo['datetime'].astype(str)

# --- ÉTAPE 1 : Standardiser les deux fichiers en UTC ---

# 1. Traitement du fichier VÉLO (Déjà en UTC +00:00)
# On le convertit en datetime object (Aware) puis on retire l'info de timezone.
df_velo['datetime'] = pd.to_datetime(df_velo['datetime'], utc=True) \
                        .dt.tz_localize(None)

# 2. Traitement du fichier MÉTÉO (Naive)
# HYPOTHÈSE : Ta météo est enregistrée à l'heure locale (Europe/Paris)
df_meteo['datetime'] = pd.to_datetime(df_meteo['datetime'])

# a. On lui donne une timezone locale ('Europe/Paris')
#    (Les arguments ambiguous/nonexistent gèrent les changements d'heure d'été)
df_meteo['datetime'] = df_meteo['datetime'].dt.tz_localize('Europe/Paris', 
                                                           ambiguous='NaT', 
                                                           nonexistent='NaT')

# b. On convertit cette heure locale en Heure de référence (UTC)
df_meteo['datetime'] = df_meteo['datetime'].dt.tz_convert('UTC')

# c. On retire l'information de timezone pour forcer l'égalité des colonnes (Naive)
df_meteo['datetime'] = df_meteo['datetime'].dt.tz_localize(None)


# --- ÉTAPE 2 : Fusion (maintenant que les colonnes sont alignées en UTC Naive) ---
df_fusionne = pd.merge(
    df_meteo,
    df_velo,
    on='datetime',
    how='inner',
    suffixes=('_meteo', '_velo')
)

print(f"✅ Synchronisation réussie ! Taille du DataFrame fusionné : {len(df_fusionne)}")

✅ Synchronisation réussie ! Taille du DataFrame fusionné : 49788


In [42]:
def exporter_donnees(df, nom_fichier="data/meteo_clean.csv"):
    print(f" Sauvegarde en cours vers '{nom_fichier}'...")
    
    # --- Option 1 : La version Standard (US/International) ---
    # Parfait si tu réutilises ce fichier en Python plus tard
    # df.to_csv(nom_fichier) 
    
    # --- Option 2 : La version "Excel Français" (Recommandée) ---
    # Pour être sûr que les accents passent et que les colonnes soient bien séparées
    df.to_csv(
        nom_fichier,
        sep=',',             # On utilise le point-virgule (standard Excel FR)
        decimal=',',         # On utilise la virgule pour les décimales (ex: 20,5 au lieu de 20.5)
        encoding='utf-8-sig',# 'utf-8-sig' force Excel à bien lire les accents (é, à, è)
        index=True           # CRUCIAL : On garde l'index (tes dates !)
    )
    
    print("Export réussi !")

exporter_donnees(df_fusionne, 'export_meteo_velo.csv')

 Sauvegarde en cours vers 'export_meteo_velo.csv'...
Export réussi !
